In [24]:
import struct
import numpy as np
import keras
from keras.utils import to_categorical
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D , MaxPooling2D , Activation
from keras.layers import Dense , Dropout , Flatten
from keras.layers import BatchNormalization
from keras import backend as K

#extract dataset from ubyte files
def read_idx(filename):
    with open(filename , 'rb') as f:
        zero , data_type , dims = struct.unpack('>HBB' , f.read(4))
        shape = tuple(struct.unpack('>I' , f.read(4))[0] for d in range(dims))
        return np.frombuffer(f.read() , dtype = np.uint8).reshape(shape)
    

In [25]:
x_train = read_idx('C:\\Users\\Zahid\\Desktop\\Deep Learning\\fashion_mnist\\train-images-idx3-ubyte')
y_train = read_idx('C:\\Users\\Zahid\\Desktop\\Deep Learning\\fashion_mnist\\train-labels-idx1-ubyte')
x_test = read_idx('C:\\Users\\Zahid\\Desktop\\Deep Learning\\fashion_mnist\\t10k-images-idx3-ubyte')
y_test = read_idx('C:\\Users\\Zahid\\Desktop\\Deep Learning\\fashion_mnist\\t10k-labels-idx1-ubyte')

In [26]:
print('Number of samples' , len(x_train))
print('Number of labels ' , len(y_train))
print('Images Dimensions' , x_train[0].shape)
print('Label Dimensions' , y_train.shape)


img_rows = x_train[0].shape[0]
img_cols = x_train[0].shape[1]

x_train = x_train.reshape(x_train.shape[0] , img_rows , img_cols , 1)
x_test = x_test.reshape(x_test.shape[0] , img_rows , img_cols , 1)

print(x_train.shape)
print(x_test.shape)

Number of samples 60000
Number of labels  60000
Images Dimensions (28, 28)
Label Dimensions (60000,)
(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [27]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print(x_train.shape)
print(x_test.shape)

#one hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

num_classes = y_test.shape[1]
print(num_classes)
num_pixels = x_train.shape[1] * x_train.shape[2]
print(num_pixels)

input_shape = (img_rows , img_cols , 1)

(60000, 28, 28, 1)
(10000, 28, 28, 1)
10
784


In [29]:
#Creating Model

model = Sequential()

#two sets of CRP
model.add(Conv2D(20 , kernel_size = (5,5), input_shape = input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2) , strides = (2,2)))

model.add(Conv2D(50 , kernel_size = (5,5)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2) , strides = (2,2)))


model.add(Flatten())

model.add(Dense(500 , activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes , activation='softmax'))

model.compile(loss = 'categorical_crossentropy' , optimizer = keras.optimizers.Adadelta() , metrics =['accuracy'])

print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 24, 24, 20)        520       
_________________________________________________________________
activation_7 (Activation)    (None, 24, 24, 20)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 20)        80        
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 12, 12, 20)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 8, 8, 50)          25050     
_________________________________________________________________
activation_8 (Activation)    (None, 8, 8, 50)          0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 8, 50)         

In [ ]:
batch_size = 64
epochs = 10

final_model = model.fit(x_train , y_train , batch_size = batch_size , epochs = epochs , verbose = 1 , validation_data = (x_test , y_test) )#verbose how much information to see when training

score = model.evaluate(x_test , y_test , verbose = 1)
print("Test loss:" , score[0])
print("Test accuracy:" , score[1])

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 275s 5ms/step - loss: 0.4761 - acc: 0.8335 - val_loss: 0.3469 - val_acc: 0.8748
Epoch 2/10
60000/60000 [==============================] - 233s 4ms/step - loss: 0.3171 - acc: 0.8851 - val_loss: 0.3108 - val_acc: 0.8864
Epoch 3/10
60000/60000 [==============================] - 225s 4ms/step - loss: 0.2777 - acc: 0.9004 - val_loss: 0.5619 - val_acc: 0.8559
Epoch 4/10
 1664/60000 [..............................] - ETA: 3:57 - loss: 0.2268 - acc: 0.9123